In [6]:
from google.oauth2 import service_account
from google.cloud import kms
from cryptography.fernet import Fernet

import firebase_admin
from firebase_admin import auth, credentials, firestore


Global params

In [7]:
# SERVICE_ACCOUNT_FILE = '../firebase-adminsdk.json'
KMS_SERVICE_ACCOUNT_FILE = '../../kms-admin.json'

project_id = 'wallet-login-45c1c'

KMS params

In [8]:
# Build the parent name from the project and location.
parent = f"projects/{project_id}/locations/global"

# Build the key ring name.
key_ring_name = f"{parent}/keyRings/gladius-key-ring"

# Build the key name.
key_name = f"{key_ring_name}/cryptoKeys/gladius-key"

print(key_name)

projects/wallet-login-45c1c/locations/global/keyRings/gladius-key-ring/cryptoKeys/gladius-key


#### Connect to KMS

In [9]:

# Create the credentials object from the service account file.
kms_credentials = service_account.Credentials.from_service_account_file(KMS_SERVICE_ACCOUNT_FILE)

# Create the client with the credentials.
kms_client  = kms.KeyManagementServiceClient(credentials=kms_credentials)

#### Connect to Firebase

In [11]:
if not firebase_admin._apps:
    firebase_cred = credentials.Certificate('../../firebase-adminsdk.json')
    firebase_admin.initialize_app(firebase_cred)
db = firestore.client()

In [13]:
user_id = 'WUTqVpvvvcaroORmoC34HwweJJp1'  # Test03
user_ref = db.collection('users').document(user_id)
encrypted_private_key = user_ref.get().get('encrypted_stellar_secret')
encrypted_encryption_key = user_ref.get().get('encrypted_encryption_key')


In [14]:
response = kms_client.decrypt(
    request={"name": key_name, "ciphertext": encrypted_encryption_key}
)
encryption_key = response.plaintext


In [16]:
f = Fernet(encryption_key)

decrypted_private_key = f.decrypt(encrypted_private_key)
decrypted_private_key

b'GCRXQGTZK6Y6G7MIC4K4IUKWYSF5J5KICIUDLIV5SHY3WXG5SVSGAUU6'